In [ ]:
import glob
import time
import numpy as np
np.set_printoptions(suppress=True)

from SmoothSparseWeights_CVXOPT import *

In [ ]:
exampleName="cheburashka"
poses_folder="poses-2"
poses_ind=7
clip_k=4
handle_num=11


# exampleName="cow"
# poses_folder="poses-2"
# poses_ind=5
# clip_k=4
# handle_num=15


# exampleName="wolf"
# poses_folder="poses-1"
# poses_ind=2
# clip_k=4
# handle_num=21



weights = {'W_spatial': 1.0}

In [ ]:
#### read obj file

### rest_poses:
vertices_obj_path="./models/"+exampleName+"-all/"+exampleName+".obj"
mesh0=TriMesh.FromOBJ_FileName(vertices_obj_path)

### other poses
vertices_obj_dir="./models/"+exampleName+"-all/"+poses_folder+"/"
filenames=glob.glob(vertices_obj_dir+"*.obj")
print(filenames)

mesh1_list=[]
for i in range(len(filenames)):
    mesh1_list.append(TriMesh.FromOBJ_FileName(filenames[i]))


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 


In [ ]:
#### read our output of simplex_hull.py 
results_path="./results_pipeline/"+exampleName+"-"+str(handle_num)+"/result.txt"
endmembers, mixing_weights=load_result(results_path)
### endmembers shape is handle * poses * 12??

# ### this is to make sure the 3*4 or 4*3 error does not happen.
# endmembers=endmembers.reshape((-1, 4, 3))
# endmembers=np.transpose(endmembers, (0,2,1))
# endmembers=endmembers.reshape((len(mixing_weights), -1, 12))


endmembers=endmembers.reshape((len(mixing_weights),-1))
mixing_weights=mixing_weights.T


print (endmembers.shape)
print (mixing_weights.shape)
print (mixing_weights[:3])
mixing_weights_clip, grad_zero_indices=clip_first_k_values(mixing_weights, clip_k)
print (mixing_weights_clip[:3])
print ((col_major(mixing_weights_clip)[grad_zero_indices]).sum()) ### should be zero

In [ ]:
scale=find_scale(vertices1)
print (scale)

In [ ]:
recovered_vertices2=recover_vertices(mixing_weights, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

In [ ]:
recovered_vertices2=recover_vertices(mixing_weights_clip, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

initials={'x': col_major(mixing_weights_clip.reshape((-1,len(endmembers)))).reshape((-1,1))}
print (initials['x'].shape)

start=time.time()

res=run(mesh0, mesh1_list, weights, endmembers, mixing_weights.ravel(), grad_zero_indices, initials=initials)

res=res.reshape(mixing_weights.shape)
end=time.time()
print("using time: ", end-start)

recovered_vertices2=recover_vertices(res, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

In [ ]:
print (res.reshape((-1, len(endmembers)))[:3]) ### should have same zeros as initial input.
print ((col_major(res)[grad_zero_indices]).sum()) ### should be zero
print ((col_major(res)[grad_zero_indices]).max()) ### should be zero

In [ ]:
print (((1-res.sum(axis=-1))**2).sum()/len(res))
print (res.min(), res.max())

In [ ]:
recover_diff= (res.dot(endmembers)-mixing_weights.dot(endmembers))
recover_diff= (recover_diff**2).sum()
print (recover_diff)

In [ ]:
# #### read gt per vertex transformation 
# groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+"-*.DMAT")
# print(groundtruth_names)

# gt_all=None
# for i in range(len(groundtruth_names)):
#     gt=load_DMAT(groundtruth_names[i])
#     gt=gt.T
#     gt=gt.reshape((-1,4,3))
#     gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
#     if gt_all is None:
#         gt_all=gt
#     else:
#         gt_all=np.hstack((gt_all, gt))
    
# print(gt_all.shape)


In [ ]:
#### read gt per vertex mixing weights 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+".DMAT")
print(groundtruth_names)
gt=load_DMAT(groundtruth_names[0])
gt_mixing_weights=gt.T.copy()
print (gt_mixing_weights.shape)

print (((gt_mixing_weights.sum(axis=-1)-1)**2).sum())

In [ ]:
### load groundtruth endmember
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.Tmat")
print(groundtruth_names)

gt_handles=None
for i in range(len(groundtruth_names)):
    gt=load_Tmat(groundtruth_names[i])
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_handles is None:
        gt_handles=gt
    else:
        gt_handles=np.hstack((gt_handles, gt))
    
print(gt_handles.shape)
gt_handles=gt_handles.reshape(len(groundtruth_names), -1, 12)

gt_handles=np.transpose(gt_handles.reshape(-1, len(groundtruth_names), 12), (1,0,2)) ### is this true?

print(gt_handles.shape)

for i in range(len(gt_handles)):
    print (gt_handles[i][2].reshape((3,4)))

test=gt_handles[1][2].reshape((3,4))[:,:3]
print (test.T.dot(test))

In [ ]:
### plot handles corresponding to gt mixing weights, recovered mixing weights, and post-processed mixing weights.
###

eps=1e-10

# Vertices=vertices2[:,poses_ind*3:poses_ind*3+3]

## use recovered vertices instead.
recovered_vertices2=recover_vertices(res, vertices1, endmembers).reshape((len(vertices1),-1))
Vertices=recovered_vertices2[:,poses_ind*3:poses_ind*3+3]


gt_handle_position=gt_mixing_weights.T.dot(Vertices)/(gt_mixing_weights.T.sum(axis=-1)[...,np.newaxis]+eps)
recovered_handle_position=mixing_weights.T.dot(Vertices)/(mixing_weights.T.sum(axis=-1)[...,np.newaxis]+eps)
cliped_handle_position=mixing_weights_clip.T.dot(Vertices)/(mixing_weights_clip.T.sum(axis=-1)[...,np.newaxis]+eps)
postprocessed_handle_position=res.T.dot(Vertices)/(res.T.sum(axis=-1)[...,np.newaxis]+eps)


%matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig= plt.figure()
ax=fig.add_subplot(111, projection='3d')

ax.scatter(gt_handle_position[:,0],gt_handle_position[:,1], gt_handle_position[:,2], c='k', s=50)
ax.scatter(recovered_handle_position[:,0],recovered_handle_position[:,1], recovered_handle_position[:,2], c='r', s=50)
# ax.scatter(cliped_handle_position[:,0],cliped_handle_position[:,1], cliped_handle_position[:,2], c='g', s=50)
ax.scatter(postprocessed_handle_position[:,0],postprocessed_handle_position[:,1], postprocessed_handle_position[:,2], c='b', s=50)
ax.scatter(Vertices[:,0],Vertices[:,1], Vertices[:,2], color='y', s=1, edgecolors='none')

plt.show()